# Exploratory analysis on weighted undirected movie graph

1. Read movie: [all the users that rated this movie] file. (Input)
2. Create weighted undirected graph G(V,E).
3. V = movie_title and E = is the two users rated the same movie.
4. E weight = Jaccard index = (intersection of users that rated i and j)/(union of users that rated i and j)
5. Find communities using Fast Greedy Newman algorithm based on movie genres.

The weighted undirected G(V,E) has edges between movies that denote the users that rated both the movies out of all the users combined that rated either of these movies. This measure of similarity/dissimilarity is called Jaccard index. Hence, higher value of this index would indicate that the two movies have been rated by pretty much the same set of users with respect to the pool of users that have rated either of these movies. However, we can't infer anything about how well a movie would do since the graph doesn't take into account the rating to them by the users. However, by performing community detection, we can identify movies that have been rated by similar set of users on the basis of their genres. So, this would give us the most popular genres in that community.

import required libraries

In [33]:
library(doMC)
library(Kmisc)
library(igraph)
library(data.table)
registerDoMC(4)

building the weighted undirected movie graph

In [34]:
#set path
setwd("C:/Users/VishankBhatia/Desktop/FellowshipAI/MovieRecomSys")

#reading weighted undirected movie graph data
x <-read.table(file="./FINAL/data/wu_movie_graph.txt", header = FALSE, sep = "\t", quote="", dec = ".")

#building weighted undirected movie user graph
colnames(x) = c("node_1", "node_2", "weight")
wu_graph <- graph.data.frame(x, directed=FALSE)

#graph properties
cat("The number of nodes: ", vcount(wu_graph))
cat("\nThe number of edges: ", ecount(wu_graph))

#removes the loop and/or multiple edges
movie_wu_graph <- simplify(wu_graph)

The number of nodes:  1663
The number of edges:  968622

performing community detection on the above graph

In [35]:
#running community detection on the network
moviesNetCommunity <- fastgreedy.community(movie_wu_graph, weights =  E(movie_wu_graph)$weight)

#read movie - genre file and process it
genreList = fread("./FINAL/data/movie_genre.txt", sep = "\t", header = FALSE)

#separate into two strings
Encoding(genreList$V1) = 'latin1'
genreList$V1 = iconv(genreList$V1, "latin1", "ASCII", sub="")
genreList$V1 = gsub("^\\(", "", genreList$V1)
genreList$V1 = gsub("\\([^[:digit:]]+\\)", "", genreList$V1)
genreList$V1 = gsub("^\\s+|\\s+$", "", genreList$V1)
genreList$V2 = gsub("^\\(", "", genreList$V2)
genreList$V2 = gsub("^\\s+|\\s+$", "", genreList$V2)

#save processed movie - genre file
write.table(genreList, file = "./FINAL/data/processed_genre.txt",sep = "\t", row.names = FALSE)

creating a movie: genre dictionary from the above list

In [36]:
#create a movie: genre dictionary from the above list
genreDict <- list()
for (i in 1: length(genreList[[1]])){
  genreDict[genreList[[1]][i]] = genreList[[2]][i]
}

finding the popular genres within communities

In [37]:
#finding the popular genres within communities
commNumber <- sort(unique(moviesNetCommunity$membership))
print(sizes(moviesNetCommunity))
cat("------------------------------------------------------------------------------------\n")
cat("Genre frequency is:")
print(table(unlist(genreDict)))

for (i in commNumber) {
  currCommMovies <- V(movie_wu_graph)[which(moviesNetCommunity$membership == i)]
  commGenres <- genreDict[currCommMovies]
  genreSummary <- table(unlist(commGenres))
  cat("------------------------------------------------------------------------------------\n")
  cat("Genre summary for community ",i)
  print(genreSummary)
  cat("------------------------------------------------------------------------------------\n")
  #applying threshold to actually find the most popular genres in that community
  cat("Threshold for community ")
    print(length(commGenres) * 0.1)
  cat("------------------------------------------------------------------------------------\n")
  cat("Genre summary after applying threshold for community ",i)
  print(genreSummary[which(genreSummary > threshold)])
cat("\n\n\n")
}

Community sizes
  1   2   3 
 10 766 887 
------------------------------------------------------------------------------------
Genre frequency is:
                 action   adventure   animation    children      comedy 
          2          32          18           2          35         248 
      crime documentary       drama     fantasy   film-noir      horror 
         29          46         498          15           6          64 
    musical     mystery     romance      sci-fi    thriller         war 
         41          23         203          61         243          70 
    western 
         27 
------------------------------------------------------------------------------------
Genre summary for community  1
adventure    comedy     drama   romance  thriller 
        1         1         3         3         2 
------------------------------------------------------------------------------------
Threshold for community [1] 1
--------------------------------------------------------

computing community rating, k-nearest neighbour rating and all neighbour rating and comparing all of them to the actual rating

In [38]:
#pre-processing on the movie - rating file 
ratings = fread("./FINAL/data/movie_avg_rating.txt", header = FALSE, sep = "\t",encoding="Latin-1")
Encoding(ratings$V1) = 'latin1'
ratings$V1 = gsub("^\\(", "", ratings$V1)
ratings$V1 = gsub("^\\s+|\\s+$", "", ratings$V1)
ratings$V1 = gsub("^\\s+|\\s+\\s$", "", ratings$V1)

#save to and read from file
write.table(ratings,"./FINAL/data/movie_avg_rating_processed.txt",row.names = FALSE, sep='\t',fileEncoding="latin1")
processedRatings <- fread("./FINAL/data/movie_avg_rating_processed.txt",sep = "\t")

#creating a dictionary of movie - avg. rating
ratingDict <- list()
for (i in 1: length(processedRatings[[1]])){
  ratingDict[[ processedRatings[[1]][i] ]] <- processedRatings[[2]][i]
}

getting vertex IDs of some example movies

In [39]:
#the movies for which we want to predict rating
vertexId1 <- which(V(movie_wu_graph)$name == "Toy Story (1995)")
vertexId2 <- which(V(movie_wu_graph)$name == "I Love Trouble (1994)")
vertexId3 <- which(V(movie_wu_graph)$name == "It Takes Two (1995)")

getting the communities these example movies belong to

In [40]:
#find the community that these movies belong to
Comm1 = moviesNetCommunity$membership[vertexId1]
Comm2 = moviesNetCommunity$membership[vertexId2]
Comm3 = moviesNetCommunity$membership[vertexId3]
movies <- c("Toy Story (1995)", 
            "I Love Trouble (1994)",
            "It Takes Two (1995)")
community <- c(Comm1, Comm2, Comm3)
actualRatings = c(3.88,2.50,2.67)

In [41]:
community

[1] 2 3 3

computing community rating, nearest neighbor rating and all neighbors rating; comparing it with actual rating of the example movies

In [42]:
k = 10
#iterating over all three movies
for (i in 1:3) {
  #get the rating of the community they belong to
  commOfMovie <- community[i]
  vertices <- V(movie_wu_graph)[which(moviesNetCommunity$membership == commOfMovie)]
  commRatings = ratingDict[vertices]
  commRatings = commRatings[which(commRatings != 0)]
  
  #get the rating of k-nearest neigbhors, here k = 10
  edgeNodes = x[which(x$"node_1" == movies[i] | x$"node_2" == movies[i]),]
  edgeNodes =  edgeNodes[order(edgeNodes$"weight"),]
  topKNeighbours = data.frame(edgeNodes[1:k,])
  topKNeighbours$"weight" = NULL
  nearestNeighbours <- c()
  for (j in 1:k){
    nearestNeighbours <- c(nearestNeighbours,topKNeighbours[j,which((topKNeighbours[j,])!=movies[i])])
  }
  nearestNeighbourRatings = ratingDict[nearestNeighbours]
  nearestNeighbourRatings <- unlist(nearestNeighbourRatings)
  nearestNeighbourRatings = nearestNeighbourRatings[which(nearestNeighbourRatings != 0)]
  
  #get the rating of all neighbours  
  neighBour = unlist(neighbors(movie_wu_graph, movies[i]))
  neighbourRatings = ratingDict[neighBour]
  neighbourRatings = neighbourRatings[which(neighbourRatings != 0)]
  
  allRatings = c(as.numeric(unlist(commRatings)), as.numeric(unlist(nearestNeighbourRatings)), as.numeric(unlist(neighbourRatings)))
  cat(movies[i], "actual rating:",actualRatings[i],"\n")
  cat("Community Rating ", mean(as.numeric(unlist(commRatings))), "\n")
  cat("Nearest Neighbor Rating", mean(as.numeric(unlist(nearestNeighbourRatings))), "\n")
  cat("Neighbors Rating", mean(as.numeric(unlist(neighbourRatings))), "\n")
  cat("All Combined ", mean(allRatings), "\n")
  cat("\n")
}
#same community rating if they belong to the same community!

Toy Story (1995) actual rating: 3.88 
Community Rating  3.295444 
Nearest Neighbor Rating 2.983891 
Neighbors Rating 3.101941 
All Combined  3.164432 

I Love Trouble (1994) actual rating: 2.5 
Community Rating  2.887347 
Nearest Neighbor Rating 2.808951 
Neighbors Rating 3.144714 
All Combined  3.032863 

It Takes Two (1995) actual rating: 2.67 
Community Rating  2.887347 
Nearest Neighbor Rating 3.078599 
Neighbors Rating 3.131115 
All Combined  3.006762 



Here, we see that community rating is more accurate out of all. So, performing community detection on the movie network on the basis of the genre each movie belongs to has turned out to be a better predictor than just looking at k-neighbours or all the neighbours of a node in the movie network.